# MPI4py Hands On

## Hello MPI

General Note: The following examples are all small (low computational burden). Hence we are allowed to run on login nodes.

As a initial test, try to run the following python script (named helloMPI.py)

In [ ]:
from mpi4py import MPI
# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
# Print a message from each process
print(f"Hello from process {rank} of {size}")

On a terminal, run the program as:

If everything is installed properly, you should see as output: (If not, make sure you loaded the proper conda env in your terminal)

So here we started two python processes, each process printing simply its ID (rank) and total number of processes started. No IPC so far

## Basic Communication Patterns

General Note: We will only use numpy arrays to do Inter-Process-Communication. This is the most common use-case in HPC and DataScience. But you can in principle exchange any python data between processes. (Less efficient than numpy arrays, since data needs to be pickled/serialized under the hood, whereas the numpy array can be treated like a C-pointer)

### Point-to-Point

We send a 1D numpy array (10 items, datatype float64) from process 0 to process 1

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# automatic MPI datatype discovery
if rank == 0:
    data = np.arange(10, dtype=np.float64)
    comm.Send(data, dest=1, tag=13)
    print ("0 sent:", data)
elif rank == 1:
    data = np.empty(10, dtype=np.float64)
    comm.Recv(data, source=0, tag=13)
    print ("1 recv:", data)

Run with two processes

#### _Exercise_

Send a 2d array (size 3x3, datatype int) from rank 0 to rank 1 (In productive applications, these would typically be detector images)

_Hint_: You can treat metadata (i.e. image size) as hard coded (global)

_Proposed solution_

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

imageSize = 3

if rank == 0:
    #We emulate a (small) detector image, typically rank 0 would load it from file
    detImage_2BeSent = np.arange(imageSize**2, dtype=np.int64).reshape(imageSize,imageSize)
    comm.Send(detImage_2BeSent, dest=1, tag=13)
    print ("0 sent:\n", detImage_2BeSent)
elif rank == 1:
    data = np.empty(imageSize**2, dtype=np.int64)
    comm.Recv(data, source=0, tag=13)
    detImage_Recv = data.reshape(imageSize,imageSize)
    print ("1 recvd:\n", detImage_Recv)

#### _Exercise_

_Proposed solution_

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

dataTag = 13
metaDataTag = 14

if rank == 0:
    #We emulate a (small) detector image, typically rank 0 would load it from file
    #Hence only rank 0 knows its size (might be varying). Datatype is assumed fixed
    imageSize = 3 #this information would e.g be derived from hdf5 metadata of image directly
    #imageSize = 50 #Deadlocks, but this is MPI-implementation dependent
    detImage_2BeSent = np.arange(imageSize**2, dtype=np.int64).reshape(imageSize,imageSize)
    #Send data first
    comm.Send(detImage_2BeSent, dest=1, tag=dataTag)
    print ("0 sent:\n", detImage_2BeSent)
    #Send Meta data (we could also send an int here, but for a non quadratic image would be an array)
    comm.Send(np.array(imageSize, dtype=np.int64), dest=1, tag=metaDataTag)

elif rank == 1:
    #Receive Metadata first (We must know how large the buffer is that we allocate
    imageSize_asArray = np.empty(1, dtype=np.int64)
    comm.Recv(imageSize_asArray, source=0, tag=metaDataTag)
    imageSize = imageSize_asArray[0]
    #Now that we know what to expect we can allocate receive buffer
    data = np.empty(imageSize**2, dtype=np.int64)
    comm.Recv(data, source=0, tag=dataTag)
    detImage_Recv = data.reshape(imageSize, imageSize)
    print ("1 recvd:\n", detImage_Recv)

_Question_: Why is this code working? The `Send` and `Recv` are blocking, but the sequence of `Send`s does not match the Sequence of `Recv`s (order is reversed!)

## Blocking versus Non-Blocking Communication

### What means blocking? Why Deadlocks depend on data size

When you use blocking sends (MPI.Send) and blocking receives (MPI.Recv) in MPI, the order of operations matters, especially for large data. 

If the order of MPI.Recv calls on one process does not match the order of MPI.Send calls on the other, a deadlock can occur. Here’s why:
How Blocking Communication Works

    Blocking Send (MPI.Send): The call does not return until the message buffer can be reused (i.e., the message is either copied out or the matching receive is ready).
    Blocking Receive (MPI.Recv): The call does not return until the message is received.

Why Small Data Works

For small messages, MPI implementations often use eager protocol:

    The data is copied into a temporary buffer and sent immediately.
    The send call returns as soon as the data is buffered, even if the receive is not yet posted.
    This can hide mismatched order for small messages.

Why Large Data Fails

For large messages, MPI typically uses rendezvous protocol:

    The send call waits until the matching receive is posted before transferring data.
    If the receive order is wrong, both processes can end up waiting for each other, causing a deadlock

### Making blocking/non-blocking more explicit explicit

Take away from above and motivation:

Blocking Send (MPI.Send): The call does not return until the  <span style="color:blue">send message buffer can be reused</span>.

Note that nothing is said about the state of the receiver. 
This is confusing or at least vague

And: <span style="color:red">This is also mean</span>: If you test your MPI-application with a tiny dummy dataset on the login node, all works well,
if you then submit your large real-world job to the cluster, it deadlocks)

But this is how it is defined in the MPI standard. (And the standard is always and by defintion correct)

#### Synchronous Send and Receive

You can use `comm.Ssend` to enforce synchronous send. `comm.Ssend` waits for a hand shake with the corresponding receiver, only afterwards it returns.

#### _Exercise_
Edit the above script and replace a single `comm.Send` by a `comm.Ssend` such that the application deadlocks for sure (also for small datasets)

#### Aynchronous Send and Receive
You can use `comm.Isend` to enforce asynchronous (or non-blocking) send
`comm.Isend` should be used tother with `comm.Irecv`, the non-blocking version of `comm.Recv`

Example:

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # Non-blocking send
    data = np.asarray([1, 2, 3])
    req_send = comm.Isend(data, dest=1, tag=77)
    # Do some computation while the send is in progress
    result = 42**2
    # Wait for the send to complete
    req_send.Wait()
    print ("Sent:", data)
elif rank == 1:
    #Allocate memory for the receive buffer
    data = np.empty(3, np.int64)
    # Non-blocking receive
    req_recv = comm.Irecv(data, source=0, tag=77)
    # Do some computation while the receive is in progress
    result = 100/26
    # Wait for the receive to complete
    req_recv.Wait()
    print("Received:", data)

The above example also highlights that non-blocking calls can be used to interleave communication with calculation.

This might give a performance increase as additional benefit. But this is not in the focus here. (The focus is on correct code)

#### _Exercise_

Modify the above script such that it never deadlocks, no matter what the sequence of sends/receives is and no matter how big data is

#### _Proposed Solution_

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()


## Basic Communication Patterns Continued

### Broadcast
As the name suggest, comm.BroadCast is used to fant out data from one rank to all the others.

The broadcasting rank is typially rank zero, whereas all other ranks > 0 recieve data

Example (taken from official documentation)

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

#Rank 0 allocates and initializes/define data to be broadcast
if rank == 0:
    data = np.arange(100, dtype='i')
#All other ranks allocate receive buffer
else:
    data = np.empty(100, dtype='i')
#Note 'collective' operation (*every* rank calls Bcast, not just rank 0) 
comm.Bcast(data, root=0)
#Check for all ranks that they have received the Broadcaster's data
for i in range(100):
    assert data[i] == i

### Scatter and Gather

#### _Exercise_

Implement parrallel matrix multiplication Ax for a (16 x 7) matrix A and a (7 x 1) vector x using broadcast scatter and gather.
The code should work for two cores only.

At the end, rank 0 (who collects the results) makes sure that the parallel version is correct (by doing the calculation itself)

#### _Proposed solution_

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()


### Anonymous receive
MPI.any..
Exercise: The first n-1 rank send data the the last ranks which recieives all (while number of received messages < n-1...

## Integration with Slurm
So far all examples were launched from command line and executed on login node.

Real world jobs (HPC use case with large memory consumption and and intense number crunching) must be submitted to the cluster.